## Week 3 Peer Graded Assignment - Segmenting and Clustering Neighborhoods in Toronto
Karen McEwen \
February 2020

### Assignment - Part I: 
1) Write code to scrape Wikipedia page for postal codes in Canada \
2) transform data into a pandas dataframe \
3) To create the above dataframe:\
    - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood \
    - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. \
    - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table. \
    - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park. \
    - Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making. \
    - In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe. \
4) submit a link to your notebook

In [1]:
# import packages we will need to scrape the data - will use beautiful soup to scrape the data
!pip install beautifulsoup4
from bs4 import BeautifulSoup
!pip install lxml

In [2]:
# import other packages we may need
import pandas as pd
import numpy as np
import requests
import time
print('packages imported')

packages imported


In [3]:
# url for the data is https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
# assign the link of the website to scrape the data and assign it to variable named website_url using a get request
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
print('Website has been requested')

Website has been requested


In [4]:
#turn the HTML into a beautiful soup text object
soup = BeautifulSoup(website_url,'html.parser')
#print(soup.prettify())


Looking at the raw data, and the wikipedia page developer's tools, the table begins at  table class="wikitable sortable" with the table headings Postcode, Borough, and Neighbourhood

In [5]:
My_table = soup.find('table',{'class':'wikitable sortable'})
#My_table

### First glance at scraped data
The Postcode, Borough, and Neighbourhood data is within \<td> html tags, with some of the neighborhoods and all of the boroughs also having links which are within \<a> tags.  \
Some of the postal codes are not assigned.  \
Boroughs have more than one postal code.  \
All but one of the neighborhoods has unique postal codes, with the exception of St. James Town in Downtown Toronto which has two postal codes.\
Queen's Park is listed as a borough with no neighborhoods.  According to the instructions, we will give it a neighborhood of the same name.(There is also a Queen's Park that is a neighborhood of Toronto.) \
Runnymede is a neighborhood associated with two boroughs. \
Several postal codes have multiple neighborhoods in them, such as M1C which has one borough and 3 neighborhoods. 
One 'neighborhood' is a postal box Stn A PO Boxes 25 The Esplanade and another is the Business Reply Mail Processing Centre 969 Eastern. Mississauga is not actually a borough of Toronto but a nearby city where the mail is processed.

### Taking the scraped data and putting it into a pandas dataframe

In [6]:
# find all the table rows
my_table_rows = My_table.find_all('tr')
#my_table_rows

In [7]:
# find the table data \<td> in the table rows
for tr in my_table_rows:
    td=tr.find_all('td')
    for i in td:
        row = i.text
       # print(row)


We now have a way to get all of the table data, listed by rows.  We need to put this data into a pandas dataframe. We'll use the same for loop above, but put the data into the empty lists of Postalcode, Borough, and Neighbourhood that we created above.

In [8]:
# find the table data \<td> in the table rows and append it to the Postalcode, Borough, and Neighborhood lists
# create empty lists where the data will go.
Postalcode = []
Borough = []
Neighborhood = []
for tr in my_table_rows:
    counter =1
    td=tr.find_all('td')
    for i in td:
        row = i.text
        if counter == 1:
            Postalcode.append(row)
        elif counter == 2:
            Borough.append(row)
        elif counter == 3:
            Neighborhood.append(row)
        else:
            end
        counter +=1
   
        

In [9]:
# now we have all the data into separate lists
#print(Postalcode)
#print(Borough)
#print(Neighborhood)

In [10]:
# now put the lists into a dictionary 
toronto_dictionary = {'Postcode':Postalcode, 'Borough':Borough, "Neighbourhood":Neighborhood}
#toronto_dictionary

In [11]:
df_toronto = pd.DataFrame.from_dict(toronto_dictionary)
#df_toronto.head()

Now that we have a dataframe, we need to remove any values that are not assigned.  We also need to remove extraneous characters like \[] and \\n.  Also, we want to make sure that all the postcodes are unique to neighborhoods, by putting multiple neighborhoods together.

In [12]:
# removing rows that are 'not assigned' from Borough column (we'll leave the Neighbourhood column alone for now)

cond1 = df_toronto['Borough']!='Not assigned'
df_toronto_postcode = df_toronto[cond1]
#df_toronto_postcode.head()

In [13]:
# remove the trailing newline characters from Neighborhood
df_toronto_postcode = df_toronto_postcode.replace('\n','', regex=True)
#df_toronto_postcode

In [14]:
# fix Borough of Queen's Park
print(df_toronto_postcode[df_toronto_postcode['Borough'] == 'Queen\'s Park'])


  Postcode       Borough Neighbourhood
9      M9A  Queen's Park  Not assigned


In [15]:
#df.loc[df['First_name'] == 'Bill', 'name_match'] = 'Match'
df_toronto_postcode.loc[df_toronto_postcode['Borough'] == 'Queen\'s Park', 'Neighbourhood'] = 'Queen\'s Park'
print(df_toronto_postcode[df_toronto_postcode['Borough'] == 'Queen\'s Park'])

  Postcode       Borough Neighbourhood
9      M9A  Queen's Park  Queen's Park


In [16]:
# are there any other neighborhoods that need to be renamed?
print(df_toronto_postcode[df_toronto_postcode['Borough'] == 'Not Assigned'])

Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []


In [17]:
#let's see how our dataframe looks
df_toronto_postcode.describe(include="all")

,Postcode,Borough,Neighbourhood
count,210,210,210
unique,103,11,207
top,M9V,Etobicoke,Queen's Park
freq,8,44,2


There are no more neighborhoods that are not assigned.  We now have a dataframe of scraped data with 210 entries and 3 columns.

In [18]:
#use groupby to see how many boroughs and neighbourhoods are in each postal code
df_toronto_postcode.groupby('Postcode', as_index=False).count()

,Postcode,Borough,Neighbourhood
0,M1B,2,2
1,M1C,3,3
2,M1E,3,3
3,M1G,1,1
4,M1H,1,1
...,...,...,...
98,M9N,1,1
99,M9P,1,1
100,M9R,4,4
101,M9V,8,8


In [19]:
# Many postcodes had more than one Neighborhood in them. 
# Now we want to list all the neighborhoods into their postal code so that the postal code is only listed once
# use groupby method and aggregate with a comma
df_t_code = df_toronto_postcode.groupby('Postcode', as_index=False).agg(lambda x: ', '.join(x))
df_t_code.head()

,Postcode,Borough,Neighbourhood
0,M1B,"Scarborough, Scarborough","Rouge, Malvern"
1,M1C,"Scarborough, Scarborough, Scarborough","Highland Creek, Rouge Hill, Port Union"
2,M1E,"Scarborough, Scarborough, Scarborough","Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Now we have the Neighbourhoods aggregated as well as the Boroughs.  We want to split off the Boroughs so there is only one listed in each column.

#### Using str.split(): (from geeksforgeeks.org)
Pandas <code>str.split() method can be applied to a whole series. .str has to be prefixed everytime before calling this method to differentiate it from the Python’s default function otherwise, it will throw an error. \
Syntax: Series.str.split(pat=None, n=-1, expand=False) \


In [20]:
#new data frame with split value column for Borough
new_df=df_t_code['Borough'].str.split(',',n=-1,expand=True)
new_df

,0,1,2,3,4,5,6,7
0,Scarborough,Scarborough,None,None,None,None,None,None
1,Scarborough,Scarborough,Scarborough,None,None,None,None,None
2,Scarborough,Scarborough,Scarborough,None,None,None,None,None
3,Scarborough,None,None,None,None,None,None,None
4,Scarborough,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
98,York,None,None,None,None,None,None,None
99,Etobicoke,None,None,None,None,None,None,None
100,Etobicoke,Etobicoke,Etobicoke,Etobicoke,None,None,None,None
101,Etobicoke,Etobicoke,Etobicoke,Etobicoke,Etobicoke,Etobicoke,Etobicoke,Etobicoke


Now we have a new dataframe with just the separated Boroughs.  We want to grab just the first column (labelled '0') \
and replace the original 'Borough' column with it in the df_t_code dataframe

In [21]:
#now we need to get rid of the duplicated Boroughs
#first lets make a copy of the dataframe
df_t_code2=df_t_code.copy(deep=True)
df_t_code2


,Postcode,Borough,Neighbourhood
0,M1B,"Scarborough, Scarborough","Rouge, Malvern"
1,M1C,"Scarborough, Scarborough, Scarborough","Highland Creek, Rouge Hill, Port Union"
2,M1E,"Scarborough, Scarborough, Scarborough","Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,"Etobicoke, Etobicoke, Etobicoke, Etobicoke","Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,"Etobicoke, Etobicoke, Etobicoke, Etobicoke, Et...","Albion Gardens, Beaumond Heights, Humbergate, ..."


In [22]:
# When using iloc or loc, always need to include the place setter for rows first (here it is : for all rows)
df_t_code2['newBorough']=new_df.iloc[:,0]
df_t_code2.head()

,Postcode,Borough,Neighbourhood,newBorough
0,M1B,"Scarborough, Scarborough","Rouge, Malvern",Scarborough
1,M1C,"Scarborough, Scarborough, Scarborough","Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Scarborough, Scarborough, Scarborough","Guildwood, Morningside, West Hill",Scarborough
3,M1G,Scarborough,Woburn,Scarborough
4,M1H,Scarborough,Cedarbrae,Scarborough


In [23]:
# now we drop the original column 'Borough'
df_t_code2.drop(['Borough'], axis=1, inplace=True)
df_t_code2.head()

,Postcode,Neighbourhood,newBorough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough


In [24]:
# now move the columns around and rename them
df_t_code2=df_t_code2[['Postcode','newBorough','Neighbourhood']]
df_t_code2.head()

,Postcode,newBorough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [25]:
# let's rename the columns
df_t_code2 = df_t_code2.rename(columns = {'newBorough':'Borough'})
df_t_code2.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [26]:
# Save the dataframe as a csv file so we don't have to keep rerunning Part 1 for parts 2 and 3
df_t_code2.to_csv('toronto_neighborhoods.csv', index=False)

In [27]:
#In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe
df_t_code2.shape

(103, 3)